###Modificación de datos

In [0]:
file = 'dbfs:/FileStore/shared_uploads/danielmm97@gmail.com/sales.csv'

sales_df = (spark.read
            .format("csv")
            .option("header", "true")     
            .option("inferSchema", True)
            .load(file))

sales_df.printSchema()

sales_df.show(5)

root
 |-- Order_ID: integer (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Ship_Date: date (nullable = true)

+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+
| Order_ID|Order_Date|    Item_Type|Units_Sold|Unit_Price|              Region|             Country| Ship_Date|
+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+
|535113847|2014-10-08|       Snacks|       934|    152.58|Middle East and N...|          Azerbaijan|2014-10-23|
|874708545|2015-02-22|    Cosmetics|      4551|     437.2|Central America a...|              Panama|2015-02-27|
|854349935|2015-12-09|       Fruits|      9986|      9.33|  Sub-Saharan Africa|Sao Tome and Prin...|2016-01-18|

In [0]:
# withColumn(): Añade una nueva columna al Dataframe

from pyspark.sql.functions import lit, expr

sales_df.withColumn("Sent", lit(False)).show(5)   #  lit() hace que contenga un valor constante en todas las filas

sales_df.withColumn("Total_Price", expr("Units_Sold *  Unit_Price")).show(5)   # Añade el resultado de ese cálculo


# withColumnRenamed(): cambia nombre de una columna

sales_df.withColumnRenamed("Region","Logistics_Area").show(5)

+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+-----+
| Order_ID|Order_Date|    Item_Type|Units_Sold|Unit_Price|              Region|             Country| Ship_Date| Sent|
+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+-----+
|535113847|2014-10-08|       Snacks|       934|    152.58|Middle East and N...|          Azerbaijan|2014-10-23|false|
|874708545|2015-02-22|    Cosmetics|      4551|     437.2|Central America a...|              Panama|2015-02-27|false|
|854349935|2015-12-09|       Fruits|      9986|      9.33|  Sub-Saharan Africa|Sao Tome and Prin...|2016-01-18|false|
|892836844|2014-09-17|Personal Care|      9118|     81.73|  Sub-Saharan Africa|Sao Tome and Prin...|2014-10-12|false|
|129280602|2010-02-04|    Household|      5858|    668.27|Central America a...|              Belize|2010-03-05|false|
+---------+----------+-------------+----------+---------

In [0]:
# drop(): elimina una o varias columnas del Dataframe.

sales_df.withColumn("Total_Price", expr("Units_Sold *  Unit_Price"))

resumen_df = sales_df.drop('Unit_Price','Region')

resumen_df.show(5)


# dropna(): elimina filas que contengan valores nulos o faltantes.
    # how: indica si se debe eliminar una fila si contiene algún valor nulo o faltante (any) o sólo si todos los valores son nulos (all).
    # thresh: indica el número mínimo de valores válidos que debe tener una fila para que no sea eliminada.
    # subset: aplicar a sólo a una(s) columna(s)

not_null_df = sales_df.dropna(how='all', thresh=5)   # Prueba

not_null_df.show()

+---------+----------+-------------+----------+--------------------+----------+
| Order_ID|Order_Date|    Item_Type|Units_Sold|             Country| Ship_Date|
+---------+----------+-------------+----------+--------------------+----------+
|535113847|2014-10-08|       Snacks|       934|          Azerbaijan|2014-10-23|
|874708545|2015-02-22|    Cosmetics|      4551|              Panama|2015-02-27|
|854349935|2015-12-09|       Fruits|      9986|Sao Tome and Prin...|2016-01-18|
|892836844|2014-09-17|Personal Care|      9118|Sao Tome and Prin...|2014-10-12|
|129280602|2010-02-04|    Household|      5858|              Belize|2010-03-05|
+---------+----------+-------------+----------+--------------------+----------+
only showing top 5 rows

+---------+----------+---------------+----------+----------+--------------------+--------------------+----------+
| Order_ID|Order_Date|      Item_Type|Units_Sold|Unit_Price|              Region|             Country| Ship_Date|
+---------+----------+-----

In [0]:
# cast(): junto con withColumn() y col() cambia el tipo de dato de una columna.

from pyspark.sql.functions import col

sales_df.withColumn('Order_ID', col('Order_ID').cast('string')).printSchema()


root
 |-- Order_ID: string (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Ship_Date: date (nullable = true)



In [0]:
# Pasar a MAY; min

from pyspark.sql.functions import upper, lower

sales_df.withColumn('region', upper(col('region'))).show(5)

+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+
| Order_ID|Order_Date|    Item_Type|Units_Sold|Unit_Price|              region|             Country| Ship_Date|
+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+
|535113847|2014-10-08|       Snacks|       934|    152.58|MIDDLE EAST AND N...|          Azerbaijan|2014-10-23|
|874708545|2015-02-22|    Cosmetics|      4551|     437.2|CENTRAL AMERICA A...|              Panama|2015-02-27|
|854349935|2015-12-09|       Fruits|      9986|      9.33|  SUB-SAHARAN AFRICA|Sao Tome and Prin...|2016-01-18|
|892836844|2014-09-17|Personal Care|      9118|     81.73|  SUB-SAHARAN AFRICA|Sao Tome and Prin...|2014-10-12|
|129280602|2010-02-04|    Household|      5858|    668.27|CENTRAL AMERICA A...|              Belize|2010-03-05|
+---------+----------+-------------+----------+----------+--------------------+--------------------+----

In [0]:
# Eliminar espacios: ltrim(izq); rtrim(dcha); trim(ambos)

from pyspark.sql.functions import ltrim, rtrim, trim

sales_df.withColumn('region', trim(col('region'))).show(5)

+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+
| Order_ID|Order_Date|    Item_Type|Units_Sold|Unit_Price|              region|             Country| Ship_Date|
+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+
|535113847|2014-10-08|       Snacks|       934|    152.58|Middle East and N...|          Azerbaijan|2014-10-23|
|874708545|2015-02-22|    Cosmetics|      4551|     437.2|Central America a...|              Panama|2015-02-27|
|854349935|2015-12-09|       Fruits|      9986|      9.33|  Sub-Saharan Africa|Sao Tome and Prin...|2016-01-18|
|892836844|2014-09-17|Personal Care|      9118|     81.73|  Sub-Saharan Africa|Sao Tome and Prin...|2014-10-12|
|129280602|2010-02-04|    Household|      5858|    668.27|Central America a...|              Belize|2010-03-05|
+---------+----------+-------------+----------+----------+--------------------+--------------------+----

In [0]:
# Ejemplo general:

#dataframe.write.format(args).mode(save_mode).option('key', 'value').save(path)    

# Save mode: append / overwrite / errorIfExists / ignore


###Agregaciones

In [0]:
# Se pueden llamar sobre el conjunto completo del dfa aunque el mayor uso que se le da y mayor provecho es aplicarlo a agrupaciones.

# Para formar agrupaciones por columna/s, utilizamos 'groupBy(cols)'.

# Lo más usual es a continuación realizar agregados por grupo: '.agg()' (facilita alias, varios agregados).

In [0]:
# Número de ventas realizadas por región (count())
from pyspark.sql.functions import count, countDistinct

sales_df.groupBy('Region','Country').agg(count('Item_Type').alias('NumVentas')).orderBy('Region','Country').show(5)

# Número de productos distintos (countDistinct()) vendidos por región
sales_df.select('Item_Type').distinct().show()

sales_df.groupBy('Region','Country').agg(countDistinct('Item_Type').alias('TiposArtículo')).orderBy('Region','Country').show(5)

+------+----------+---------+
|Region|   Country|NumVentas|
+------+----------+---------+
|  Asia|Bangladesh|      545|
|  Asia|    Bhutan|      534|
|  Asia|    Brunei|      558|
|  Asia|  Cambodia|      584|
|  Asia|     China|      554|
+------+----------+---------+
only showing top 5 rows

+---------------+
|      Item_Type|
+---------------+
|      Baby Food|
|         Cereal|
|           Meat|
|      Household|
|     Vegetables|
|      Beverages|
|Office Supplies|
|      Cosmetics|
|  Personal Care|
|         Fruits|
|         Snacks|
|        Clothes|
+---------------+

+------+----------+-------------+
|Region|   Country|TiposArtículo|
+------+----------+-------------+
|  Asia|Bangladesh|           12|
|  Asia|    Bhutan|           12|
|  Asia|    Brunei|           12|
|  Asia|  Cambodia|           12|
|  Asia|     China|           12|
+------+----------+-------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import col, sum, avg, stddev

#Unidades vendidas (sum()) de cada tipo de producto por región, de mayor a menor unidades:
sales_df.groupBy('Item_Type','Region').agg(sum('Units_Sold').alias('TotalUnidades')).orderBy(col('TotalUnidades').desc()).show(10,truncate=False)

# Media (avg()) y desviación típica (stddev()) de las unidades vendidas por producto y región, orden descendente media:
sales_df.groupBy('Item_Type','Region').agg(avg('Units_Sold').alias('MediaUnidades'),stddev('Units_Sold').alias('DesvTip')).orderBy(col('MediaUnidades').desc()).show(10,truncate=False)


+---------------+------------------+-------------+
|Item_Type      |Region            |TotalUnidades|
+---------------+------------------+-------------+
|Cosmetics      |Sub-Saharan Africa|11318369     |
|Baby Food      |Sub-Saharan Africa|11103311     |
|Cereal         |Europe            |11076271     |
|Beverages      |Sub-Saharan Africa|11073025     |
|Office Supplies|Sub-Saharan Africa|11022827     |
|Household      |Sub-Saharan Africa|10967644     |
|Snacks         |Sub-Saharan Africa|10916846     |
|Office Supplies|Europe            |10905284     |
|Vegetables     |Europe            |10859623     |
|Meat           |Sub-Saharan Africa|10792594     |
+---------------+------------------+-------------+
only showing top 10 rows

+-------------+---------------------------------+------------------+------------------+
|Item_Type    |Region                           |MediaUnidades     |DesvTip           |
+-------------+---------------------------------+------------------+----------------

In [0]:
# Pedidos máximos (max()) y mínimos (min()):

from pyspark.sql.functions import min, max

sales_df.groupBy('Item_Type','Region').agg(max('Units_Sold').alias('PedidoMáximo'),min('Units_Sold').alias('PedidoMínimo')).orderBy(col('PedidoMáximo').desc()).show(10,truncate=False)


+-------------+---------------------------------+------------+------------+
|Item_Type    |Region                           |PedidoMáximo|PedidoMínimo|
+-------------+---------------------------------+------------+------------+
|Cosmetics    |Sub-Saharan Africa               |10000       |1           |
|Cereal       |Sub-Saharan Africa               |10000       |2           |
|Meat         |Middle East and North Africa     |10000       |16          |
|Vegetables   |Australia and Oceania            |10000       |1           |
|Snacks       |Middle East and North Africa     |10000       |1           |
|Personal Care|Europe                           |10000       |9           |
|Household    |Sub-Saharan Africa               |10000       |1           |
|Meat         |Central America and the Caribbean|10000       |9           |
|Personal Care|Sub-Saharan Africa               |9999        |9           |
|Beverages    |Europe                           |9999        |1           |
+-----------